# Обработка датасета

## Библиотеки и скрипты

In [1]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

In [2]:
# Функция приведения типов
def astype_col(df, colgroup, coltype):
    for colname in colgroup:
        df[colname] = df[colname].astype(coltype)
    return df

# Функция удаления столбцов
def drop_columns(df, columns):
    for column in columns:
        df = df.drop(column, axis=1)  
    return df

# Функция транслитерации
def get_translite(columns):
    lst = []
    prefix = 'ST_'
    for column in columns:
        r = translit(prefix + column, 'ru', reversed=True).lower()
        r = re.sub("\'|\(|\)|:", "", r)
        r = re.sub("\ |-|‑", "_", r)
        lst.append(r)
    return lst

# Функция получение имен столбцов с индексом
def get_idx_columns(columns):
    lst = []
    for index, value in enumerate(columns):
        print(index, value)
        lst.append(index)
    print(lst)

## Обзор данных

In [3]:
# Загружаем датасет без 1-й строки и делаем переиндексацию с удалением столбца index
df = pd.read_csv('../data/wp_cn_requests_price.csv')
# Удаляем дубликаты по id_session
df = df.drop_duplicates(subset='id_session', keep="first")

In [4]:
df.head(5)

,id,id_session,id_YM,id_GA,created,location,branch,direction,amount,price
0,15,e70acf53510a6d7e46f75e16bf37a4eb,,NaN,2020-12-23 10:35:19,Yeysk,Выездное обслуживание оборудования,Спецтехника,100,800
1,19,20d25acbd2795a7359e42fadbc91cc25,,NaN,2020-12-23 10:48:21,St Petersburg,Выездное обслуживание оборудования,Торговое оборудование,2,5000
2,20,7af0528d1dc81ea5a58205ed46fd8272,,NaN,2020-12-23 10:49:58,St Petersburg,Здравоохранение и медицина,Обслуживание медицинского оборудования,5,10000
6,25,08d113256a6b43aa56933083cd4afdd1,,NaN,2020-12-23 12:13:32,Moscow,Выездное обслуживание оборудования,"Постаматы, вендинг, банкоматы",200,790
11,30,20350ef6faa93b10cb4d17be08ab6fe3,,NaN,2020-12-23 13:07:43,St Petersburg,"Управление недвижимостью, ЖКХ",Обслуживание и ремонт БЦ/ТЦ,22,1990


In [5]:
df.shape

(1205, 10)

## Обработка данных

In [6]:
# Удаление ненужных данных
columns = ['id', 'id_session']
df = drop_columns(df, columns)

# Убираем 2020 год (там одни тесты)
df = df[df['created'].str.contains('2021.*')]

In [7]:
df.columns

Index(['id_YM', 'id_GA', 'created', 'location', 'branch', 'direction',
       'amount', 'price'],
      dtype='object')

In [8]:
# Переименование столбцов
df.columns = get_translite(df.columns)
df.columns

Index(['st_id_ym', 'st_id_ga', 'st_created', 'st_location', 'st_branch',
       'st_direction', 'st_amount', 'st_price'],
      dtype='object')

In [9]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110 entries, 226 to 2513
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   st_id_ym      1110 non-null   object
 1   st_id_ga      1030 non-null   object
 2   st_created    1110 non-null   object
 3   st_location   1110 non-null   object
 4   st_branch     1110 non-null   object
 5   st_direction  1110 non-null   object
 6   st_amount     1110 non-null   int64 
 7   st_price      1110 non-null   object
dtypes: int64(1), object(7)
memory usage: 78.0+ KB


st_id_ym         0
st_id_ga        80
st_created       0
st_location      0
st_branch        0
st_direction     0
st_amount        0
st_price         0
dtype: int64

In [10]:
# Обработка выбросов
df.loc[df['st_amount'] > 2000, 'st_amount'] = df['st_amount'].median()

In [11]:
# Корректировка значений
df.loc[df['st_id_ym'] == '', 'st_id_ym'] = 'undefined'
df.loc[df['st_id_ga'] == '', 'st_id_ga'] = 'undefined'
df.loc[df['st_id_ym'].isnull(), 'st_id_ym'] = 'undefined'
df.loc[df['st_id_ga'].isnull(), 'st_id_ga'] = 'undefined'
df.loc[df['st_price'] == 'По запросу', 'st_price'] = 0
df.loc[df['st_price'] == 0.0, 'st_price'] = df['st_price'].median()

# Поиск и замена по регулярке
df.loc[df['st_location'].str.contains("’"), 'st_location'] = df['st_location'].str.replace("’", "")
df.loc[df['st_location'].str.contains(r"\\'"), 'st_location'] = df['st_location'].str.replace(r"\\'", "")

# Заменим название направления типа "Другое" на название корневой отрасли
df.loc[df['st_direction'] == 'Другое', 'st_direction'] = df['st_branch']

In [12]:
# df.loc[df['st_location'].str.contains("\'")]

In [13]:
# Приведение типов
df = astype_col(df, ['st_amount'], coltype='uint8')
df = astype_col(df, ['st_price'], coltype='float64')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110 entries, 226 to 2513
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   st_id_ym      1110 non-null   object 
 1   st_id_ga      1110 non-null   object 
 2   st_created    1110 non-null   object 
 3   st_location   1110 non-null   object 
 4   st_branch     1110 non-null   object 
 5   st_direction  1110 non-null   object 
 6   st_amount     1110 non-null   uint8  
 7   st_price      1110 non-null   float64
dtypes: float64(1), object(6), uint8(1)
memory usage: 102.7+ KB


In [15]:
df.describe()

,st_amount,st_price
count,1110.000000,1110.000000
mean,18.094595,14339.909910
std,37.918344,33305.220302
min,0.000000,0.000000
25%,3.000000,5000.000000
50%,5.000000,5000.000000
75%,10.000000,9500.000000
max,250.000000,378500.000000


In [16]:
# Скорректируем дату для более удобной группировки данных
# df['st_created'] = pd.to_datetime(df['st_created']).dt.strftime('%Y-%m-%d')

In [17]:
df.head(5)

,st_id_ym,st_id_ga,st_created,st_location,st_branch,st_direction,st_amount,st_price
226,1609586985682049430,GA1.2.934928888.1609586985,2021-01-02 14:30:42,Kazan,Мобильные сотрудники,Строительные бригады,3,5000.0
227,1609610415617206319,GA1.2.233829537.1609610416,2021-01-02 21:01:43,Barnaul,Телекоммуникации и ИТ,Выездной ИТ-сервис,5,5000.0
228,undefined,undefined,2021-01-02 23:45:30,Sergiyev Posad,"Управление недвижимостью, ЖКХ",ЖКХ сервис,3,20000.0
230,1609623797665217591,GA1.2.52594619.1609623797,2021-01-03 00:46:09,Minsk,"Управление самозанятыми, ИП и подрядчиками",Управление ИП и самозанятыми,3,5000.0
233,160964815239544789,GA1.2.1046709437.1609648153,2021-01-03 07:30:46,Barnaul,Выездное обслуживание оборудования,Кондиционирование и вентиляция,3,5000.0


In [18]:
df.shape

(1110, 8)

## Сохранение данных

In [19]:
# Сохраняем в файл
df.to_csv('../data/td_site.csv', index=False, encoding='utf-8', sep=';')